In [1]:
from keras.models import load_model
model = load_model("CNN_vgg16.keras")

c:\Users\igrac\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
c:\Users\igrac\anaconda3\lib\site-packages\keras\src\saving\saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 62 variables whereas the saved optimizer has 10 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [2]:
# obtain the dataset again
import os
import numpy as np
import pandas as pd

from const import *
from data_io import get_poster, plot_poster
from sklearn.model_selection import train_test_split

from tensorflow.keras import Input, Model, datasets
from tensorflow.keras.optimizers import Adam
import tensorflow as tf


In [3]:
def performance_metrics(y_test,predy):
    from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
    from tensorflow.keras.metrics import AUC  

    from tensorflow.keras.ops import softmax
    from sklearn.metrics import roc_curve, auc, f1_score 
    import matplotlib.pyplot as plt


    predy_class = np.argmax(predy, axis=1)
    truey_class = np.argmax(y_test, axis=1)

    confusion_matrix = confusion_matrix(truey_class, predy_class)
    accuracy = accuracy_score(truey_class, predy_class)
    precision = precision_score(truey_class, predy_class, average='macro')
    recall = recall_score(truey_class, predy_class, average='macro')
    f1 = f1_score(truey_class, predy_class, average='macro')
    print(f'Confusion Matrix: \n{confusion_matrix}')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1: {f1}')

    tf_auc = AUC(num_thresholds = 200, from_logits = True, multi_label = True)
    tf_auc.update_state(y_test, predy)
    AUC= tf_auc.result().numpy()
    print(f'AUC: {AUC}')
    
    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    # feed predy into softmax function for actual probabilities
    probi_y = softmax(predy, axis=-1)
    for i in range(n_genre):
        fpr[i], tpr[i], _ = roc_curve(y_test[:,i], probi_y[:,i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    plt.figure()
    for i in range(n_genre):
        plt.plot(fpr[i], tpr[i], label = 'ROC curve of class {0} (area = {1:0.2f})'
                                        ''.format(i, roc_auc[i]))
    plt.plot([0,1], [0,1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curve')
    plt.legend(loc = 'lower right')
    plt.show    
    plt.savefig('ROC.png', dpi=300)

    

    return None


In [26]:
movie = []
predy_genre = []
predy_prob = []
n_genre = 3
IMG_SIZE = (345, 230, 3)

testset=pd.read_csv("by_decades_movies_genres.csv")
print(testset.head())


# sample movie id
movie_index = testset['id'].tolist()
movie_genre=testset['genre'].tolist()
for i in range(len(movie_index)):
    movie.append(get_poster(movie_index[i]))
movie = np.array(movie)

predy = model.predict(movie)
predy_class = np.argmax(predy, axis=1)

predy_class.shape

for i in range(len(predy_class)):
    if predy_class[i] == 0:
        genre="Action"
        predy_prob.append(round(predy[i][0],4))
    elif predy_class[i] == 1:
        genre="Romance"
        predy_prob.append(round(predy[i][1],4))       
    elif predy_class[i] == 2:
        genre="Thriller"
        predy_prob.append(round(predy[i][2],4))
    else:
        genre="Error"
    predy_genre.append(genre)
    # if movie_genre[i]!="Action" & movie_genre[i]!="Romance" & movie_genre[i]!="Thriller":
    #     predy_genre_clean.append("/")
    # else:
    #     predy_genre_clean.append(genre)

#if movie_genre[i] not equal to "Action", "Romance" or "Thriller" predy_genre_clean.append("/")

for i in range(len(predy_class)):
    print(f'{i+1}. ID: {movie_index[i]} Predicted Label: {predy_genre[i]} Probability: {predy_prob[i]} True Label: {movie_genre[i]}')

# Convert list to DataFrame
result = {'ID': movie_index, 'Predicted Label': predy_genre, 'Probability': predy_prob, 'True Label': movie_genre}
df_test=pd.DataFrame(result)
# Save DataFrame as CSV
df_test.to_csv('vgg16_sample movie result.csv', index=False)



   Unnamed: 0       id   genre
0           0  1055982   Drama
1           1  1251266  Horror
2           2  1333580   Drama
3           3  1531575  Action
4           4  1573052   Drama
3/3 ━━━━━━━━━━━━━━━━━━━━ 18s 5s/step
1. ID: 1055982 Predicted Label: Action Probability: 0.8705999851226807 True Label: Drama
2. ID: 1251266 Predicted Label: Romance Probability: 0.7764999866485596 True Label: Horror
3. ID: 1333580 Predicted Label: Romance Probability: 0.43939998745918274 True Label: Drama
4. ID: 1531575 Predicted Label: Action Probability: 0.35850000381469727 True Label: Action
5. ID: 1573052 Predicted Label: Romance Probability: 0.8274999856948853 True Label: Drama
6. ID: 1573060 Predicted Label: Romance Probability: 0.8289999961853027 True Label: Mystery
7. ID: 1591422 Predicted Label: Romance Probability: 0.8267999887466431 True Label: Drama
8. ID: 1622401 Predicted Label: Romance Probability: 0.7767999768257141 True Label: Comedy
9. ID: 1682898 Predicted Label: Action Probability: 